# Digit Recognizer Prediction
Prediction part of the [Kaggle Digit Reconizer](https://www.kaggle.com/c/digit-recognizer/overview). 

# Setup

In [1]:
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Row

import org.apache.spark.ml.feature._
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification._

import org.apache.spark.ml.linalg._
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed._


import java.time.{Period, LocalDate, Instant}
import java.sql.Timestamp
import java.time.temporal.ChronoUnit

In [4]:
%%html
<!-- To left align the HTML components in Markdown -->
<style>
table {float:left}
</style>

<console>:60: error: missing argument list for method timed
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `timed _` or `timed(_,_)` instead of `timed`.
       timed
       ^
lastException: Throwable = null


<!-- To left align the HTML components in Markdown -->

In [3]:
val NUM_CORES = 8
val NUM_PARTITIONS = 4

spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
/*
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.driver.memory", "6g")
spark.conf.set("spark.executor.memory", "2g")
spark.conf.set("spark.master", "spark://masa:7077")
*/

Waiting for a Spark session to start...

NUM_CORES = 8
NUM_PARTITIONS = 4


<console>:60: error: missing argument list for method timed
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `timed _` or `timed(_,_)` instead of `timed`.
       timed
       ^
lastException: Throwable = null


4

In [4]:
val configMap = spark.conf.getAll.foreach(println)

(spark.serializer,org.apache.spark.serializer.KryoSerializer)
(spark.driver.host,10.115.130.67)
(spark.eventLog.enabled,true)
(spark.driver.port,36007)
(spark.hadoop.validateOutputSpecs,True)
(spark.repl.class.uri,spark://10.115.130.67:36007/classes)
(spark.jars,file:/home/oonisim/.local/share/jupyter/kernels/apache_toree_scala/lib/toree-assembly-0.3.0-incubating.jar)
(spark.repl.class.outputDir,/tmp/spark-53f3054e-a5e1-4cbb-b0b4-10cdfa9e0170/repl-cfb701e3-0c79-4bc7-a028-ff8f12078baf)
(spark.app.name,Apache Toree)
(spark.driver.memory,1g)
(spark.executor.instances,3)
(spark.history.fs.logdirectory,hdfs://oonisim:8020/logs_spark)
(spark.default.parallelism,32)
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.master,yarn)
(spark.ui.filters,org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter)
(spark.executor.memory,4g)
(spark.eventLog.dir,hdfs://oonisim:8020/logs_spark)
(spark.executor.cores,4)
(spark.driver.appUIAddress,http://10.115.130.67:4041)
(spark.org.apach

configMap: Unit = ()


## Constant

In [24]:
val PROTOCOL="hdfs://"
val DATA_DIR="/data/kaggle/mnist/data"
val MODEL_DIR="/data/kaggle/mnist/model"
val RESULT_DIR="/data/kaggle/mnist/result"

val COVERAGE:Double = 0.95

PROTOCOL = hdfs://
DATA_DIR = /data/kaggle/mnist/data
MODEL_DIR = /data/kaggle/mnist/model
RESULT_DIR = /data/kaggle/mnist/result
COVERAGE = 0.95


0.95

# MNIST Test Dataset

In [6]:
val test = spark.read.format("csv")
    .option("header", true)
    .option("inferSchema", true)
    .load(PROTOCOL + DATA_DIR + "/test.csv.gz")

test = [pixel0: int, pixel1: int ... 782 more fields]


[pixel0: int, pixel1: int ... 782 more fields]

# Dataframe for ML training
Use R formula transformation to create label and features columns for Spark ML.

In [7]:
val formula = "label ~ ."
val rformula = RFormulaModel.load(PROTOCOL + MODEL_DIR + "/kaggle_digit_recognizer_rformula.mdl")
val testDataDF = rformula
    .transform(test)
    .select("features")

formula = label ~ .
rformula = RFormulaModel(ResolvedRFormula(label=label, terms=[pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,pixel50,pixel51,pixel52,pixel53,pixel54,pixel55,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,pixel65,pixel66,pixel67,pixel68,pixel69,pixel70,pixel71,pixel72,pixel73,pixel74,pixel75,pixel76,pixel77,pixel78,pixel79,pixel80,pixel81,pixel82,pixel83,pi...


RFormulaModel(ResolvedRFormula(label=label, terms=[pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,pixel50,pixel51,pixel52,pixel53,pixel54,pixel55,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,pixel65,pixel66,pixel67,pixel68,pixel69,pixel70,pixel71,pixel72,pixel73,pixel74,pixel75,pixel76,pixel77,pixel78,pixel79,pixel80,pixel81,pixel82,pixel83,pi...

# PCA

In [8]:
val pca = PCAModel.load(PROTOCOL + MODEL_DIR + "/kaggle_digit_recognizer_pca.mdl")

pca = pca_63da76466e29


pca_63da76466e29

In [9]:
val pcaTestDataDF = pca.transform(testDataDF)

pcaTestDataDF = [features: vector, pcaFeatures: vector]


[features: vector, pcaFeatures: vector]

# Prediction

In [12]:
/*
import org.apache.spark.ml.tuning.TrainValidationSplitModel
val fitted = TrainValidationSplitModel.load(PROTOCOL + MODEL_DIR + "/kaggle_digit_recognizer_tvs.mdl")
*/

tvs = tvs_c6aadc18f7f1


tvs_c6aadc18f7f1

In [ ]:
import org.apache.spark.ml.tuning.CrossValidatorModel
val fitted = CrossValidatorModel.load(PROTOCOL + MODEL_DIR + "/kaggle_digit_recognizer_cv.mdl")

In [14]:
val predictions = fitted.transform(pcaTestDataDF)

predictions = [features: vector, pcaFeatures: vector ... 3 more fields]


[features: vector, pcaFeatures: vector ... 3 more fields]

In [15]:
predictions.show

+--------------------+--------------------+--------------------+--------------------+----------+
|            features|         pcaFeatures|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+----------+
|(784,[122,123,124...|[1612.42042408780...|[3.10440378782087...|[0.03104403787820...|       2.0|
|(784,[179,180,181...|[1919.58554625988...|[89.8726237980590...|[0.89872623798059...|       0.0|
|(784,[181,182,183...|[258.514350043858...|[3.36094066825166...|[0.03360940668251...|       9.0|
|(784,[208,210,211...|[497.736897425884...|[7.46759295209640...|[0.07467592952096...|       4.0|
|(784,[95,96,97,98...|[667.552229232708...|[2.55710185908117...|[0.02557101859081...|       3.0|
|(784,[202,203,204...|[487.203794732073...|[2.54427274091337...|[0.02544272740913...|       7.0|
|(784,[124,125,126...|[2481.24718102618...|[82.1917772496175...|[0.82191777249617...|       0.0|
|(784,[122,123,124...|[1010.70

In [23]:
predictions
    .selectExpr("cast(prediction as int) as Label")
    .coalesce(1)
    .write
    .format("csv")
    .option("header", "false")
    .mode("overwrite")
    .save(PROTOCOL + RESULT_DIR + "/kaggle_digit_recognizer_predictions.csv")

lastException: Throwable = null


In [ ]:
%bash
hadoop fs -get "${RESULT_DIR}/kaggle_digit_recognizer_predictions.csv"
cd "${RESULT_DIR}/kaggle_digit_recognizer_predictions.csv"
nl -nln -s',' part-* | tr -d ' ' > predictions.csv 